<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Example-Together" data-toc-modified-id="Example-Together-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Example Together</a></span><ul class="toc-item"><li><span><a href="#Question" data-toc-modified-id="Question-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Question</a></span></li><li><span><a href="#Considerations" data-toc-modified-id="Considerations-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Considerations</a></span></li><li><span><a href="#Loading-the-Data" data-toc-modified-id="Loading-the-Data-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Loading the Data</a></span></li><li><span><a href="#Some-Exploration-to-Better-Understand-our-Data" data-toc-modified-id="Some-Exploration-to-Better-Understand-our-Data-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Some Exploration to Better Understand our Data</a></span></li><li><span><a href="#Experimental-Setup" data-toc-modified-id="Experimental-Setup-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Experimental Setup</a></span><ul class="toc-item"><li><span><a href="#What-Test-Would-Make-Sense?" data-toc-modified-id="What-Test-Would-Make-Sense?-2.5.1"><span class="toc-item-num">2.5.1&nbsp;&nbsp;</span>What Test Would Make Sense?</a></span></li><li><span><a href="#The-Hypotheses" data-toc-modified-id="The-Hypotheses-2.5.2"><span class="toc-item-num">2.5.2&nbsp;&nbsp;</span>The Hypotheses</a></span></li><li><span><a href="#Setting-a-Threshold" data-toc-modified-id="Setting-a-Threshold-2.5.3"><span class="toc-item-num">2.5.3&nbsp;&nbsp;</span>Setting a Threshold</a></span></li></ul></li><li><span><a href="#$\chi^2$-Test" data-toc-modified-id="$\chi^2$-Test-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>$\chi^2$ Test</a></span><ul class="toc-item"><li><span><a href="#Setup-the-Data" data-toc-modified-id="Setup-the-Data-2.6.1"><span class="toc-item-num">2.6.1&nbsp;&nbsp;</span>Setup the Data</a></span></li><li><span><a href="#Calculation" data-toc-modified-id="Calculation-2.6.2"><span class="toc-item-num">2.6.2&nbsp;&nbsp;</span>Calculation</a></span></li></ul></li><li><span><a href="#Interpretation" data-toc-modified-id="Interpretation-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Interpretation</a></span></li></ul></li><li><span><a href="#Exercise" data-toc-modified-id="Exercise-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Exercise</a></span></li></ul></div>

In [1]:
import numpy as np
import pandas as pd

from scipy import stats
import seaborn as sns

# Objectives

- Conduct an A/B test in Python
- Interpret the results of the A/B tests for a stakeholder

# Example Together

## Question

We have data about whether customers completed sales transactions, segregated by the type of ad banners to which the customers were exposed.

The question we want to answer is whether there was any difference in sales "conversions" between desktop customers who saw the sneakers banner and desktop customers who saw the accessories banner in the month of May 2019.

## Considerations

What would we need to consider when designing our experiment?

Might include:

- Who is it that we're including in our test?
- How big of an effect would make it "worth" us seeing?
    - This can affect sample size
    - This can give context of a statistically significant result
- Other biases or "gotchas"

## Loading the Data

First let's download the data from [kaggle](https://www.kaggle.com/podsyp/how-to-do-product-analytics) via the release page of this repo: https://github.com/flatiron-school/ds-ab_testing/releases 

The code below will load it into our DataFrame:

In [3]:
# This will download the data from online so it can take
# some time (but relatively small download)

df = pd.read_csv('https://github.com/flatiron-school/ds-ab_testing/\
releases/download/v1.2/products_small.csv')

> Let's take a look while we're at it

In [4]:
df.head()

,Unnamed: 0,order_id,user_id,page_id,product,site_version,time,title,target
0,4122928,3e6c5e89fdddcaee0eed210ec2c9cadf,90d58d967eb72656e86059ec6f208092,2fdc16a09e0016555dd4da4a3fe84414,accessories,desktop,2019-03-06 08:42:47,banner_show,0
1,564306,feed6203517d3abf6aab13761633174b,08703dab1f004eabba25aacb7f0e5484,6b0a902b9b73d5a158d0119d6feb38ac,sneakers,mobile,2019-04-19 18:30:45,banner_show,0
2,1872289,e33d5d7941edc281646aa37763729771,bdf1d25697e21419901c94fabdafad15,9ddb7315c4357929931b48f2b3d11c62,company,mobile,2019-01-20 17:20:10,banner_show,0
3,3616779,7c4caa8d508fa7c3bbc25f35cdd9168a,8d2f23a732c9527d95678088a3bac122,1f86cd0bea31d54a5b511b42fd19401a,sneakers,mobile,2019-02-20 09:38:32,banner_show,0
4,5871482,12874b29bde8bbd43fb2b95735caf9e6,5a22604f8f31ae98ee1211ece3a02004,b533f7e2003418c63fd71471264c559a,sneakers,mobile,2019-04-24 09:19:02,banner_show,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   Unnamed: 0    1000000 non-null  int64 
 1   order_id      1000000 non-null  object
 2   user_id       1000000 non-null  object
 3   page_id       1000000 non-null  object
 4   product       1000000 non-null  object
 5   site_version  1000000 non-null  object
 6   time          1000000 non-null  object
 7   title         1000000 non-null  object
 8   target        1000000 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 68.7+ MB


## Some Exploration to Better Understand our Data

Lets's look at the different banner types:

In [6]:
df['product'].value_counts()

clothes             210996
company             203020
sneakers            201298
sports_nutrition    193200
accessories         191486
Name: product, dtype: int64

In [7]:
df.groupby('product')['target'].value_counts()

product           target
accessories       0         186127
                  1           5359
clothes           0         197720
                  1          13276
company           0         203020
sneakers          0         193375
                  1           7923
sports_nutrition  0         190363
                  1           2837
Name: target, dtype: int64

Let's look at the range of time-stamps on these data:

In [8]:
df['time'].min()

'2019-01-01 00:00:25'

In [9]:
df['time'].max()

'2019-05-31 23:59:21'

Let's check the counts of the different site_version values:

In [10]:
df['site_version'].value_counts()

mobile     718521
desktop    281479
Name: site_version, dtype: int64

In [11]:
df['title'].value_counts()

banner_show     872275
banner_click     98330
order            29395
Name: title, dtype: int64

In [12]:
len(df[df['title'] == 'order']), sum(df['target'])

(29395, 29395)

In [13]:
df.groupby('title').agg({'target': 'mean'})

,target
title,
banner_click,0
banner_show,0
order,1


## Experimental Setup

We need to filter by site_version, time, and product:

In [14]:
df_AB = df[(df['site_version'] == 'desktop') &
           (df['time'] >= '2019-05-01') &
           ((df['product'] == 'accessories') | (df['product'] == 'sneakers'))]\
.reset_index(drop=True)

In [15]:
df_AB.tail()

,Unnamed: 0,order_id,user_id,page_id,product,site_version,time,title,target
25967,5738290,ecfe9da2dbea255f05b2f526a4c663a7,cd7598ba5798ad6bd3f5a667ae4c217a,644ad50dd900d6735d8ceb15b71d50f8,accessories,desktop,2019-05-01 23:01:05,banner_show,0
25968,7885931,8c28b45ff7fbb5d5df0b7670183689db,2f28106a4db2babcfde1c4095c805632,a3d2de7675556553a5f08e4c88d2c228,sneakers,desktop,2019-05-26 11:19:09,order,1
25969,8099155,c613db2f19c7285f101196062c178490,97473368941b89c4716c4f389e9b8bd7,a3d2de7675556553a5f08e4c88d2c228,sneakers,desktop,2019-05-11 09:04:46,order,1
25970,6690689,6f7f32ecf6790e4eb06384b479f43162,73f403b4843c70c41c0db294ee479582,13f395d7c86ef276f7ce52a557fd3491,accessories,desktop,2019-05-09 09:17:24,banner_show,0
25971,7650805,af3a19fcceb477d2992f1771af3a9b34,515cb969971fb5e08ae0316f58e3f434,ef38d906f15db1efde867c68f1336946,sneakers,desktop,2019-05-20 17:58:16,banner_show,0


### What Test Would Make Sense?

Since we're comparing the frequency of conversions of customers who saw the "sneakers" banner against those who saw the "accessories" banner, we can use a $\chi^2$ test.

Note there are other hypothesis tests we can use but this should be fine since it should fit our criteria.

### The Hypotheses

$H_0$: Customers who saw the sneakers banner were no more or less likely to buy than customers who saw the accessories banner.

$H_1$: Customers who saw the sneakers banner were more or less likely to buy than customers who saw the accessories banner.

### Setting a Threshold

We'll set a false-positive rate of $\alpha = 0.05$.

## $\chi^2$ Test

### Setup the Data

We need our contingency table: the numbers of people who did or did not submit orders, both for the accessories banner and the sneakers banner. 

In [16]:
df_A = df_AB[df_AB['product'] == 'accessories']
df_B = df_AB[df_AB['product'] == 'sneakers']

In [17]:
accessories_orders = sum(df_A['target'])
sneakers_orders = sum(df_B['target'])

accessories_orders, sneakers_orders

(496, 799)

To get the numbers of people who didn't submit orders, we get the total number of people who were shown banners and then subtract the numbers of people who did make orders.

In [18]:
accessories_total = len(df_A) #number of people shown accessories banners
sneakers_total = len(df_B) #number of people who were shown sneakers banners

accessories_no_orders = accessories_total - accessories_orders 
# number of people shown banners but did not make an order
sneakers_no_orders = sneakers_total - sneakers_orders
# number of people shown banners but did not make an order

accessories_no_orders, sneakers_no_orders

(12207, 12470)

In [19]:
contingency_table = np.array([
    (accessories_orders, accessories_no_orders), 
    (sneakers_orders, sneakers_no_orders)
])

contingency_table

array([[  496, 12207],
       [  799, 12470]])

### Calculation


In [20]:
stats.chi2_contingency(contingency_table)

(60.94640624666488,
 5.86498985882812e-15,
 1,
 array([[  633.38922686, 12069.61077314],
        [  661.61077314, 12607.38922686]]))

In [22]:
chi2stat, p_value, df, expected = stats.chi2_contingency(contingency_table)

In [23]:
p_value

5.86498985882812e-15

This extremely low $p$-value suggests that these two groups are genuinely performing differently. In particular, the desktop customers who saw the sneakers banner in May 2019 bought at a higher rate than the desktop customers who saw the accessories banner in May 2019.

## Interpretation

In [26]:
contingency_table

array([[  496, 12207],
       [  799, 12470]])

In [27]:
# Find the difference in conversion rate
accessory_CR, sneaker_CR = contingency_table[:, 0] / contingency_table.sum(axis=1)

In [28]:
print(f'Conversion Rate for accessory banner:\n\t{100*accessory_CR:.3f}%')
print(f'Conversion Rate for sneaker banner:\n\t{100*sneaker_CR:.3f}%')
print('')
print(f'Absolute difference of CR: {100*(sneaker_CR-accessory_CR):.3f}%')

Conversion Rate for accessory banner:
	3.905%
Conversion Rate for sneaker banner:
	6.022%

Absolute difference of CR: 2.117%


So we can say:
- There was a statistically significant difference at the $\alpha$-level (confidence level)
- The difference was about $2.1\%$ in favor of the sneaker banner!

# Exercise

> The company is impressed with what you found and is now wondering if there is a difference in their other banner ads!

With your group, look at the same month (May 2019) but compare different platforms ('mobile' vs 'desktop') and or different banner types ('accessories', 'sneakers', 'clothes', 'sports_nutrition'). Just don't repeat the same test we did above 😉

Make sure you record what considerations you have for the experiment, what hypothesis test you performed ($H_0$ and $H_1$ too), and your overall conclusion/interpretation for the _business stakeholders_. Is there a follow-up you'd suggest? 

In [30]:
df = pd.read_csv('https://github.com/flatiron-school/ds-ab_testing/\
releases/download/v1.2/products_small.csv')

In [31]:
df.head()

,Unnamed: 0,order_id,user_id,page_id,product,site_version,time,title,target
0,4122928,3e6c5e89fdddcaee0eed210ec2c9cadf,90d58d967eb72656e86059ec6f208092,2fdc16a09e0016555dd4da4a3fe84414,accessories,desktop,2019-03-06 08:42:47,banner_show,0
1,564306,feed6203517d3abf6aab13761633174b,08703dab1f004eabba25aacb7f0e5484,6b0a902b9b73d5a158d0119d6feb38ac,sneakers,mobile,2019-04-19 18:30:45,banner_show,0
2,1872289,e33d5d7941edc281646aa37763729771,bdf1d25697e21419901c94fabdafad15,9ddb7315c4357929931b48f2b3d11c62,company,mobile,2019-01-20 17:20:10,banner_show,0
3,3616779,7c4caa8d508fa7c3bbc25f35cdd9168a,8d2f23a732c9527d95678088a3bac122,1f86cd0bea31d54a5b511b42fd19401a,sneakers,mobile,2019-02-20 09:38:32,banner_show,0
4,5871482,12874b29bde8bbd43fb2b95735caf9e6,5a22604f8f31ae98ee1211ece3a02004,b533f7e2003418c63fd71471264c559a,sneakers,mobile,2019-04-24 09:19:02,banner_show,0


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   Unnamed: 0    1000000 non-null  int64 
 1   order_id      1000000 non-null  object
 2   user_id       1000000 non-null  object
 3   page_id       1000000 non-null  object
 4   product       1000000 non-null  object
 5   site_version  1000000 non-null  object
 6   time          1000000 non-null  object
 7   title         1000000 non-null  object
 8   target        1000000 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 68.7+ MB


In [33]:
df['site_version'].value_counts()

mobile     718521
desktop    281479
Name: site_version, dtype: int64

In [38]:
df.groupby('site_version')['target'].value_counts()

site_version  target
desktop       0         265760
              1          15719
mobile        0         704845
              1          13676
Name: target, dtype: int64

In [40]:
df_May=df[(df['time'] >= '2019-05-01')]

In [41]:
df_May.head()

,Unnamed: 0,order_id,user_id,page_id,product,site_version,time,title,target
19,4880034,c59e23bca9e3e74b76e5c89591920741,885540374a1549d83561b2a40313ff2e,503567af06c98967c3cc1b97886f9a36,clothes,desktop,2019-05-15 13:40:11,banner_show,0
21,5368605,544a8db198f2c93f9efe020d0ccacb47,6882b941b57ca76302c9b7bceac19586,7f6a5bebd21c5bb052219ea59c756435,company,mobile,2019-05-23 20:33:13,banner_click,0
27,7899150,af5361b16580e2589e80d7c5e3ed0a3c,afea2091e55fc90f8b09923a2a71705f,40d319b24b329442675ea5c06f53dd29,sneakers,mobile,2019-05-21 16:46:47,banner_show,0
32,8064199,f909eb9bbf2795337dd38323b7951705,5a96585c3db94eb2ae40e559cbd2e164,94b6ea2316b773afa20925d9b79d5bf8,accessories,desktop,2019-05-19 09:26:29,banner_show,0
33,8394794,0287fbf70d2d13fb50a082af56925baa,d3921696f5f6d83a4fa112dde7b3b380,6338719e3f54583dff8d010e6673776e,sneakers,mobile,2019-05-28 19:30:57,banner_show,0


In [42]:
df_May.groupby('site_version')['target'].value_counts()

site_version  target
desktop       0          64616
              1           3224
mobile        0         180996
              1           2939
Name: target, dtype: int64

In [46]:
desktop_orders = 3224
desktop_no_orders = 64616
mobile_orders = 2939
mobile_no_orders = 180996

In [43]:
df_desktop = df_May[df_May['site_version'] == 'desktop']
df_mobile = df_May[df_May['site_version'] == 'mobile']
df_desktop

,Unnamed: 0,order_id,user_id,page_id,product,site_version,time,title,target
19,4880034,c59e23bca9e3e74b76e5c89591920741,885540374a1549d83561b2a40313ff2e,503567af06c98967c3cc1b97886f9a36,clothes,desktop,2019-05-15 13:40:11,banner_show,0
32,8064199,f909eb9bbf2795337dd38323b7951705,5a96585c3db94eb2ae40e559cbd2e164,94b6ea2316b773afa20925d9b79d5bf8,accessories,desktop,2019-05-19 09:26:29,banner_show,0
38,7560076,97808cfd3d44fa7e8cfeb71bb5f02590,1e2db5d01051fc65fffc6c27f34400a3,48c8e3f287d814493cff887ff445f535,sneakers,desktop,2019-05-03 05:34:46,banner_show,0
39,5350229,8fe80a8fef852c4291cd0be2ee7aa13f,59b65b765ce496cfe47d350486eb703f,c90f8d6de6b59e2d11c1b7483652c35b,sports_nutrition,desktop,2019-05-19 18:38:05,banner_show,0
57,8222315,cc974cfcb6c706d8188b3a89f6c25a4c,3cfb9460c5f6a7fb6be3f2bdaaa8ef56,38bf27c81ccd630a2a50cf0e166d5b06,company,desktop,2019-05-12 10:29:35,banner_click,0
...,...,...,...,...,...,...,...,...,...
999951,7160307,dbd36203dbf0aaee2c41920bb133294e,445485262fa191919fcf10829cd6e269,c47aaf1a1f20741c20d3a7a1597a776f,clothes,desktop,2019-05-03 14:08:53,banner_show,0
999963,1184549,372d5ddf14dfc917273aa4840731511f,7a708804113fc91af31b26ccaa47ff21,4d6338e82bab7964dd9ea9d4cbda5c8b,sports_nutrition,desktop,2019-05-06 11:15:58,banner_show,0
999985,8197096,9b90203a458a19bec09cd4fb6f9dc69f,bff8aee79342f86d1346d94651e84a64,876a5dd1c118fa00690f852d9ed6ad6d,company,desktop,2019-05-22 06:30:48,banner_show,0
999992,7735711,c40020044e86083b53c48ec23b1eda2a,fbb6f9203a0845ab8a824855582059c3,eeae53a3fe2725c7e1d57abed2b116cb,clothes,desktop,2019-05-02 16:00:12,banner_show,0


In [44]:
df_mobile

,Unnamed: 0,order_id,user_id,page_id,product,site_version,time,title,target
21,5368605,544a8db198f2c93f9efe020d0ccacb47,6882b941b57ca76302c9b7bceac19586,7f6a5bebd21c5bb052219ea59c756435,company,mobile,2019-05-23 20:33:13,banner_click,0
27,7899150,af5361b16580e2589e80d7c5e3ed0a3c,afea2091e55fc90f8b09923a2a71705f,40d319b24b329442675ea5c06f53dd29,sneakers,mobile,2019-05-21 16:46:47,banner_show,0
33,8394794,0287fbf70d2d13fb50a082af56925baa,d3921696f5f6d83a4fa112dde7b3b380,6338719e3f54583dff8d010e6673776e,sneakers,mobile,2019-05-28 19:30:57,banner_show,0
34,6365447,ae031dd8b5684d4a13e12f9fbca8ca6e,0e2135ead33f549ba2b774f6422c09b5,637a5f326a15510ca050600d47541060,sports_nutrition,mobile,2019-05-20 17:39:52,banner_show,0
40,5971896,19bda32538714d4e515e1f2209fd1298,88c2c9e57454f3f7e088c98442f64b58,3fc1a6f8f9d1eb67ef2f242d136923c9,sports_nutrition,mobile,2019-05-16 23:17:18,banner_show,0
...,...,...,...,...,...,...,...,...,...
999954,2385332,790b3a86be8def144804afdf71965595,1e3f776145210629e5645e621e032b48,45d39d405e9c25291a3abb02c788b99d,accessories,mobile,2019-05-21 23:34:26,banner_show,0
999960,7259328,bf36a673b94e91b6ded51931b8394e31,0f5aea8c6a5b89f12841867d152ddbfb,3167412835e4db6f876e3b15ef88edf2,company,mobile,2019-05-17 14:28:17,banner_show,0
999969,1305114,81a675ab9565e80aa4dcb65c1d12eb30,6c6604560d13cfd00453dc0623b39a17,063ac2781d3efff273b3f73ab7478c94,clothes,mobile,2019-05-03 13:33:53,banner_show,0
999980,8159492,f49debd6ec37e7dc963cc8f9263a040f,10167ef9d0127d225909c4ece2cdfc28,6b668b223896c8683f14d20d332b607e,sports_nutrition,mobile,2019-05-21 16:50:47,banner_show,0


In [45]:
type(df_May)

pandas.core.frame.DataFrame

In [47]:
contingency_table_b = np.array([
    (desktop_orders, desktop_no_orders), 
    (mobile_orders, mobile_no_orders)
])

In [48]:
contingency_table_b

array([[  3224,  64616],
       [  2939, 180996]])

In [49]:
stats.chi2_contingency(contingency_table_b)

(2063.9893282896182,
 0.0,
 1,
 array([[  1660.60140999,  66179.39859001],
        [  4502.39859001, 179432.60140999]]))

In [50]:
desktop_CR, mobile_CR = contingency_table_b[:, 0] / contingency_table.sum(axis=1)

In [51]:
print(f'Conversion Rate for desktop site:\n\t{100*desktop_CR:.3f}%')
print(f'Conversion Rate for mobile site:\n\t{100*mobile_CR:.3f}%')
print('')
print(f'Absolute difference of CR: {100*(desktop_CR-mobile_CR):.3f}%')

Conversion Rate for desktop site:
	25.380%
Conversion Rate for mobile site:
	22.149%

Absolute difference of CR: 3.230%
